In [2]:
import cv2
import os

# === Load Models ===
face_detector = cv2.FaceDetectorYN.create(
    "face_detection_yunet_2023mar.onnx", "", (320, 320),
    score_threshold=0.9, nms_threshold=0.3, top_k=5000
)
face_recognizer = cv2.FaceRecognizerSF.create("face_recognition_sface_2021dec.onnx", "")

# === Load Known Faces ===
known_features = []
known_labels = []
base_path = r"C:\Users\veyadav\Documents\OneDrive - Capgemini\Documents\INTUDO\python\notebook\faces"  # Folder with subfolders per identity

for name in os.listdir(base_path):
    person_dir = os.path.join(base_path, name)
    if not os.path.isdir(person_dir):
        continue
    for img_file in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue

        face_detector.setInputSize((img.shape[1], img.shape[0]))
        _, faces = face_detector.detect(img)
        if faces is not None and len(faces) > 0:
            aligned = face_recognizer.alignCrop(img, faces[0])
            feature = face_recognizer.feature(aligned)
            known_features.append(feature)
            known_labels.append(name)

# === Load Video ===
cap = cv2.VideoCapture(r'C:\Users\veyadav\Downloads\Drupal-Branching.mp4')
screen_width, screen_height = 1280, 720

cv2.namedWindow("DNN Face Recognition", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("DNN Face Recognition", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    face_detector.setInputSize((frame.shape[1], frame.shape[0]))
    _, faces = face_detector.detect(frame)

    if faces is not None:
        for i in range(faces.shape[0]):
            box = faces[i, :4].astype(int)

            aligned_face = face_recognizer.alignCrop(frame, faces[i])
            feature = face_recognizer.feature(aligned_face)

            best_score, best_label = float("inf"), "Unknown"
            for stored_feature, name in zip(known_features, known_labels):
                score = face_recognizer.match(stored_feature, feature, cv2.FaceRecognizerSF_FR_COSINE)
                if score < best_score:
                    best_score = score
                    best_label = name

            label_text = f"{best_label} ({best_score:.2f})" if best_score < 0.363 else "Unknown"

            # === Optional: Trigger event on match ===
            if best_score < 0.363:
                print(f"✅ Match: {best_label} | Score: {best_score:.2f} | Frame: {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}")

            # Draw label and box
            x, y, w, h = box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label_text, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # === Resize frame to fit screen ===
    fh, fw = frame.shape[:2]
    scale = min(screen_width / fw, screen_height / fh)
    new_w, new_h = int(fw * scale), int(fh * scale)
    resized_frame = cv2.resize(frame, (new_w, new_h))

    canvas = cv2.copyMakeBorder(
        resized_frame,
        top=(screen_height - new_h) // 2,
        bottom=(screen_height - new_h + 1) // 2,
        left=(screen_width - new_w) // 2,
        right=(screen_width - new_w + 1) // 2,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )

    cv2.imshow("DNN Face Recognition", canvas)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

✅ Match: dan | Score: 0.10 | Frame: 35
✅ Match: mukesh | Score: 0.02 | Frame: 35
✅ Match: dan | Score: -0.12 | Frame: 35
✅ Match: dan | Score: 0.10 | Frame: 36
✅ Match: mukesh | Score: 0.02 | Frame: 36
✅ Match: dan | Score: -0.12 | Frame: 36
✅ Match: dan | Score: 0.10 | Frame: 37
✅ Match: mukesh | Score: 0.02 | Frame: 37
✅ Match: dan | Score: -0.12 | Frame: 37
✅ Match: dan | Score: 0.10 | Frame: 38
✅ Match: mukesh | Score: 0.02 | Frame: 38
✅ Match: dan | Score: -0.12 | Frame: 38
✅ Match: dan | Score: 0.10 | Frame: 39
✅ Match: mukesh | Score: 0.02 | Frame: 39
✅ Match: dan | Score: -0.12 | Frame: 39
✅ Match: dan | Score: 0.10 | Frame: 40
✅ Match: mukesh | Score: 0.02 | Frame: 40
✅ Match: dan | Score: -0.12 | Frame: 40
✅ Match: dan | Score: 0.10 | Frame: 41
✅ Match: mukesh | Score: 0.02 | Frame: 41
✅ Match: dan | Score: -0.12 | Frame: 41
✅ Match: dan | Score: 0.10 | Frame: 42
✅ Match: mukesh | Score: 0.02 | Frame: 42
✅ Match: dan | Score: -0.12 | Frame: 42
✅ Match: dan | Score: 0.10 | Fra